In [1]:
import pandas as pd

from master_thesis.core.utils.reproducibility import seed_everything
from master_thesis.core.models.llama import load_model_and_tokenizer
from master_thesis.core.utils.prompts import load_prompt
from master_thesis.core.models.evaluate import evaluate_model_accuracy


DEVICE = "cuda"
PROMPT_TYPE = "few_shot"
POSITIVE_TOKEN = "true"
NEGATIVE_TOKEN = "false"
MODEL = "LLAMA_2_7B_CHAT"

DATA_DIR = "../../../data"
DATASETS_DIR = f"{DATA_DIR}/datasets/base_experiments/cebab"

In [2]:
seed_everything()

In [3]:
model, tokenizer = load_model_and_tokenizer(MODEL)

/home/mateusz_politycki/master-thesis-probing/.venv/lib/python3.10/site-packages/transformers/models/auto/tokenization_auto.py:732: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(
/home/mateusz_politycki/master-thesis-probing/.venv/lib/python3.10/site-packages/transformers/models/auto/auto_factory.py:468: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(
The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.
`low_cpu_mem_usage` was None, now set to True since model is quantized.


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [4]:
TOKENS = [tokenizer.encode(NEGATIVE_TOKEN)[-1], tokenizer.encode(POSITIVE_TOKEN)[-1]]

### Test

In [5]:
label_prompt = load_prompt(
    DATA_DIR,
    dataset_path="base_experiments/cebab",
    prompt_type=PROMPT_TYPE,
    prompt_aspect="review_aspect",
)

In [6]:
test = pd.read_csv(f"{DATASETS_DIR}/test/test.csv")

In [7]:
seed_everything()
evaluate_model_accuracy(model, tokenizer, test, label_prompt, "review", TOKENS, DEVICE)

100%|██████████| 1689/1689 [11:27<00:00,  2.46it/s]


0.8827708703374778

### Train

In [9]:
for aspect in ["food", "ambiance", "noise", "service"]:
    aspect_prompt = load_prompt(
        DATA_DIR,
        dataset_path="base_experiments/cebab",
        prompt_type=PROMPT_TYPE,
        prompt_aspect=f"{aspect}_aspect",
    )
    seed_everything()
    df = pd.read_csv(f"{DATASETS_DIR}/train/{aspect}_aspect.csv")
    accuracy = evaluate_model_accuracy(
        model, tokenizer, df, aspect_prompt, "label", TOKENS, DEVICE
    )
    print(f"Accuracy for {aspect}: {accuracy}")

100%|██████████| 200/200 [01:24<00:00,  2.37it/s]


Accuracy for food: 0.89


100%|██████████| 200/200 [01:25<00:00,  2.33it/s]


Accuracy for ambiance: 0.83


100%|██████████| 200/200 [01:24<00:00,  2.36it/s]


Accuracy for noise: 0.645


100%|██████████| 200/200 [01:25<00:00,  2.35it/s]

Accuracy for service: 0.865
